In [18]:
from collections import defaultdict

import numpy as np
import pandas as pd
from Predictor import Predictor
import MDP
from constants import REGION_LABEL, REGION_CODE_INVERSE, POLICY_MAP, REGION_CODE

## Load Data

In [2]:
data = pd.read_csv('./data/region_monthly_electricity_consumption.csv').rename(columns={"Unnamed: 0": "region"})

In [3]:
data.head()

,region,2005-01-01,2005-02-01,2005-03-01,2005-04-01,2005-05-01,2005-06-01,2005-07-01,2005-08-01,2005-09-01,...,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01
0,Overall,418.1,408.3,446.9,496.2,496.8,519.7,497.7,489.3,477.7,...,449.7,463,457.3,415.9,384.1,394.9,423.7,491,482.9,464.1
1,Central Region,485.4,459,500.2,566.4,573.3,590.1,568.3,551.7,539.5,...,505.4,523.1,527.6,481.4,442.7,436.7,478.9,553.5,558.8,524.8
2,Bishan,433.6,420.5,479.2,513.5,516.3,545,522.6,507.7,505.8,...,463,495.7,471.5,429.1,379.6,416.1,471.4,498,485.4,489.9
3,Bukit Merah,320,264.9,292.4,358.7,354.2,350,358.4,337.8,343.2,...,368.9,364.2,384.9,346.4,341.6,301.1,330.2,389.9,421.9,380.2
4,Bukit Timah,963.4,930.5,1035.2,1149.5,1132.3,1173.7,1101.2,1051.2,1039,...,911.5,951,916.9,834.9,797.2,760.7,876.8,1012.1,938.6,856.5


## Train Time Series Predictor

In [4]:
predictor = Predictor(data)

16:05:42 - cmdstanpy - INFO - Chain [1] start processing
16:05:42 - cmdstanpy - INFO - Chain [1] done processing
16:05:42 - cmdstanpy - INFO - Chain [1] start processing
16:05:42 - cmdstanpy - INFO - Chain [1] done processing
16:05:42 - cmdstanpy - INFO - Chain [1] start processing
16:05:42 - cmdstanpy - INFO - Chain [1] done processing
16:05:42 - cmdstanpy - INFO - Chain [1] start processing
16:05:42 - cmdstanpy - INFO - Chain [1] done processing
16:05:42 - cmdstanpy - INFO - Chain [1] start processing
16:05:42 - cmdstanpy - INFO - Chain [1] done processing
16:05:42 - cmdstanpy - INFO - Chain [1] start processing
16:05:42 - cmdstanpy - INFO - Chain [1] done processing
16:05:43 - cmdstanpy - INFO - Chain [1] start processing
16:05:43 - cmdstanpy - INFO - Chain [1] done processing
16:05:43 - cmdstanpy - INFO - Chain [1] start processing
16:05:43 - cmdstanpy - INFO - Chain [1] done processing
16:05:43 - cmdstanpy - INFO - Chain [1] start processing
16:05:43 - cmdstanpy - INFO - Chain [1]

In [6]:
predictor.predict("Geylang", "2025-06")

Geylang 2025-06 -> 446.5758261288765 [434.96666666666664, 475.2]
Primary region type: residential -> State: 1


1

## Run MDP

In [7]:
P = MDP.make_transition_prob(data)

In [8]:
R_holiday = MDP.make_rewards("holiday")
R_common = MDP.make_rewards("common")

In [9]:
mdp_common = MDP.MDP(P, R_common, 0.6)
mdp_holiday = MDP.MDP(P, R_holiday, 0.6)

In [10]:
policy_holiday = mdp_holiday.get_policy()
policy_common = mdp_common.get_policy()

In [11]:
policy_holiday

(0, 0, 0, 1, 1, 1, 0, 2, 2, 2, 2, 2)

In [12]:
policy_common

(0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2)

## Optimize

In [29]:
def vote(region: str, code: int, policy: tuple):
    region_break = REGION_LABEL[region]
    power_demand = code % 3
    action_map = defaultdict(float)
    for region_type in region_break:
        action_code = policy[REGION_CODE[region_type] * 3 + power_demand]
        action_map[action_code] += region_break[region_type]
    return max(action_map, key=action_map.get)

In [30]:
for region in REGION_LABEL:
    code = predictor.predict(region, "2024-12")
    dominant_region_type = REGION_CODE_INVERSE[code // 4]
    action_code = vote(region, code, policy_common)
    action = POLICY_MAP[action_code]
    print(f"{region} - {dominant_region_type}: {action}")

Central Region 2024-12 -> 479.70851351145876 [504.26666666666665, 551.5]
Primary region type: commercial -> State: 3
Central Region - residential: Increase supply
East Region 2024-12 -> 409.72116262492034 [450.03333333333336, 494.4]
Primary region type: residential -> State: 0
East Region - residential: Keep supply as this
North East Region 2024-12 -> 404.7515312164638 [414.3666666666667, 449.73333333333335]
Primary region type: residential -> State: 0
North East Region - residential: Keep supply as this
North Region 2024-12 -> 387.96674763803867 [394.76666666666665, 424.4]
Primary region type: residential -> State: 0
North Region - residential: Keep supply as this
West Region 2024-12 -> 385.1931854206348 [392.3666666666667, 425.19999999999993]
Primary region type: residential -> State: 0
West Region - residential: Keep supply as this
Bishan 2024-12 -> 433.7488774829808 [457.1333333333333, 498.3666666666666]
Primary region type: residential -> State: 0
Bishan - residential: Keep supply